# Exploratory Data Science

Data exploration and understanding the task at hand is a fundamental step in the Machine Learning workflow.
In this notebook, we'll take an opportunity to explore the use case, data and models we'll be using.

We have been tasked with developing an application which can identify objects in static and live images. In this notebook we use a pre-trained machine learning model, and explore how it works on static photos. 

To begin, we import a range of python packages:

In [ ]:
from os import environ

import numpy as np
from onnxruntime import InferenceSession
from PIL import Image, ImageColor, ImageDraw, ImageFont

from classes import classes


print('Imported libraries')

## Import our image

In the next cell we import the image we want to test our model on.

In [ ]:
sample_image = 'sample-images/cat.jpg'
sample = Image.open(sample_image)
sample

This image shows a cat. We need to import the image as an array so the ONNX model we will use can process the image.

In [ ]:
def transform(image):
    model_image_size = (416, 416)
    boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.transpose(image_data, [2, 0, 1])
    image_data = np.expand_dims(image_data, 0)
    return image_data


def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw, nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128, 128, 128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

In [ ]:
converted_image = transform(sample)
converted_image

## Load in a model

The model we are going to use today is the Tiny YOLO v3 model, which you can download from the ONNX Model Zoo [here](https://github.com/onnx/models/tree/main/vision/object_detection_segmentation/tiny-yolov3). The model has been trained on the [COCO](https://cocodataset.org/#home) data set, and can recognise 80 types of objects. We begin by loading in the model.

The model is stored in an s3 bucket, and we connect to it using the `boto3` library. The `boto3` library was built into the object detection notebook image, which we selected from the spawner page. As such, it is already installed in our environment.

In [ ]:
import boto3

s3_endpoint_url = environ.get('AWS_S3_ENDPOINT')
s3_access_key = environ.get('AWS_ACCESS_KEY_ID')
s3_secret_key = environ.get('AWS_SECRET_ACCESS_KEY')

# enter your user ID
s3_bucket_name = 'YOUR_USER_ID'

print('Imported s3 library')

In [ ]:
s3 = boto3.client(
    's3', endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key,
)
s3.download_file(s3_bucket_name, 'tiny-yolov3-11.onnx', 'model.onnx')

print('Downloaded model.')

You should be able to see that this file has been added to your file directory on the left hand side of the screen.

Let's now use the model to run object detection on our sample image.

In [ ]:
session = InferenceSession('model.onnx')
raw_result = session.run(
    [], {'input_1': converted_image, 'image_shape': [[416, 416]]}
)
raw_result

In [ ]:
def postprocess(raw_boxes, raw_scores, raw_class_indices):
    boxes, scores, detected_classes = [], [], []
    for raw_indices in raw_class_indices[0]:
        detected_classes.append(classes[raw_indices[1]])
        scores.append(raw_scores[tuple(raw_indices)])
        indices = (raw_indices[0], raw_indices[2])
        boxes.append(raw_boxes[indices])

    return boxes, scores, detected_classes

In [ ]:
result = postprocess(*raw_result)
result

The model has returned arrays, each of which holds information about the detected objects. The information includes identifiers for the types of objects, coordinates locating the objects within the image, and detection scores, corresponding to how certain the model is about its prediction.

We can use a few functions to help us to superimpose the information in this dictionary onto the original image.

In [ ]:
def draw_boxes(image, boxes, scores, classes):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())
    class_colors = {}
    font = ImageFont.load_default()
    image_pil = Image.open(image)

    for index, class_ in enumerate(classes):
        box = boxes[index]
        display_str = f'{class_}: {int(100 * scores[index])}%'
        if class_ not in class_colors:
            class_colors[class_] = colors[hash(class_) * 8 % len(colors)]
        color = class_colors.get(class_)
        _draw_bounding_box_on_image(
            image_pil, box[0], box[1], box[2], box[3], color, font,
            display_str_list=[display_str]
        )
    return image_pil
    image_pil.show()


def _draw_bounding_box_on_image(
        image, ymin, xmin, ymax, xmax, color, font,
        thickness=4, display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    width_scaling_factor = im_width / 416
    height_scaling_factor = im_height / 416
    (left, right, top, bottom) = (
        xmin * width_scaling_factor,
        xmax * width_scaling_factor,
        ymin * height_scaling_factor,
        ymax * height_scaling_factor,
    )
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
               (left, top)], width=thickness, fill=color)

    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)
    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height

    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)], fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str, fill="black", font=font)
        text_bottom -= text_height - 2 * margin


print('Defined image display functions')

In [ ]:
draw_boxes(sample_image, *result)

Fantastic! So you've seen how we can use a pre-trained model to identify objects in images. In the next notebooks, we will deploy this model using RHODS Model Serving, which allows us to use it as part of a larger application.